Instalar

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

     |████████████████████████████████| 274 kB 5.5 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=50eefab0ba0699d6ed65646c951e1102087a82ee358e1d5759b75cfc37f8092d
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html
     |████████████████████████████████| 7.0 MB 772 kB/s 
     |████████████████████████████████| 74 kB 2.2 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 130 kB 11.5 MB/s 
     |████████████████████████████████| 145 kB 47.3 MB/s 
     |████████████████████████████████| 843 kB 48.1 MB/s 
     |███████████████████████████████

Imports

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

Cargar dataset

In [ ]:
from detectron2.data import MetadataCatalog, DatasetCatalog

dir_path = "/content/drive/MyDrive/Mask RCNN/dataset-detectron/"

def load_data(t="train"):
    if t == "train":
        with open(dir_path+"dataset.json", 'r') as file:
            train = json.load(file)["train"]
        return train
    elif t == "val":
      with open(dir_path+"dataset.json", 'r') as file:
          val = json.load(file)["val"]
    return val


for d in ["train", "val"]:
    DatasetCatalog.register(d, lambda d=d: load_data(d))
    MetadataCatalog.get(d).set(thing_classes=["Uva"])
metadata = MetadataCatalog.get("train")

Configs

In [ ]:


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("val",)
cfg.DATALOADER.NUM_WORKERS = 1
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.OUTPUT_DIR = "/content/drive/MyDrive/Mask RCNN/dataset-detectron/output"
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

Train

In [ ]:
from detectron2.engine import DefaultTrainer


trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[12/28 23:41:15 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[12/28 23:41:23 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/28 23:41:38 d2.utils.events]:  eta: 0:03:08  iter: 19  total_loss: 4.133  loss_cls: 0.6956  loss_box_reg: 0.4837  loss_mask: 0.6919  loss_rpn_cls: 1.96  loss_rpn_loc: 0.1858  time: 0.6840  data_time: 0.1419  lr: 1.6068e-05  max_mem: 1705M
[12/28 23:41:50 d2.utils.events]:  eta: 0:02:50  iter: 39  total_loss: 2.215  loss_cls: 0.6319  loss_box_reg: 0.5981  loss_mask: 0.6542  loss_rpn_cls: 0.1319  loss_rpn_loc: 0.1632  time: 0.6522  data_time: 0.0269  lr: 3.2718e-05  max_mem: 1705M
[12/28 23:42:03 d2.utils.events]:  eta: 0:02:38  iter: 59  total_loss: 2  loss_cls: 0.5345  loss_box_reg: 0.6321  loss_mask: 0.5781  loss_rpn_cls: 0.1374  loss_rpn_loc: 0.1519  time: 0.6513  data_time: 0.0042  lr: 4.9367e-05  max_mem: 1705M
[12/28 23:42:15 d2.utils.events]:  eta: 0:02:21  iter: 79  total_loss: 1.869  loss_cls: 0.4892  loss_box_reg: 0.6176  loss_mask: 0.4896  loss_rpn_cls: 0.1042  loss_rpn_loc: 0.1461  time: 0.6325  data_time: 0.0048  lr: 6.6017e-05  max_mem: 1705M
[12/28 23:42:27 d2.utils.e

Evaluacion

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

dir_path = "/content/drive/MyDrive/Mask RCNN/dataset-detectron/"
with open(dir_path+"dataset.json", 'r') as file:
  #train = json.load(file)["train"]
  val = json.load(file)["val"]
  predictor = DefaultPredictor(cfg)

from detectron2.utils.visualizer import ColorMode

preds =[]

for d in val:
  print(d["file_name"])    
  im = cv2.imread(d["file_name"])
  real_len = len(d["annotations"])
  outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
  pred_len = len(outputs["instances"])
  preds.append([d["file_name"],real_len,pred_len])
  #print("REAL NUM:",real_len," PRED NUM:",pred_len)
        
  v = Visualizer(im[:, :, ::-1],
        metadata=metadata, 
        scale=0.5, 
        instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
      )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])
        
print("Nombre  |  Cantidad Real  | Cantidad Predecida")
for i in preds:
  print(i[0],i[1],i[2])

Output hidden; open in https://colab.research.google.com to view.